In [ ]:
#|hide
import torchvision.transforms as T
import matplotlib as mpl, matplotlib.pyplot as plt
import torch.optim as optim
from fastai.vision.all import *
from torchvision.datasets import MNIST
from functools import wraps
import wandb
from fastai.callback.wandb import *
from FastFF.performance import *
from FastFF.moe import *

In [ ]:
#|hide
set_seed(0)
wandb.login()

In [ ]:
#|hide
#|eval: false

n=4
h=64
topk=4
bs = 512 

model = MoE(28*28, 10, n, topk, hidden_dim=h)
ds = get_mnist_ds()
cbs = [GetGradCB([model.experts[0][0]]), ShowGraphCallback()]
dls = DataLoaders(TfmdDL(ds[0], bs, True),TfmdDL(ds[1], bs, False))
learn = Learner(dls, model, loss_func=F.cross_entropy, metrics=accuracy, opt_func = Adam)

In [ ]:
learn.fit_one_cycle(15, lr_max=4e-3)

In [ ]:
#|eval: false
cbs[1].show(model.experts[0][0], lambda x: x.abs().mean((1,2))[:,None])

In [ ]:
#|hide
#|eval: false
d, l = 2, 64
topk = 4

model = FFF(28*28, 10, d, topk, hidden_dim=l)
cbs = [FFFLeavesDistCB('leaf_distr'), GetGradCB(['weight'], model.gate[0]), ShowGraphCallback()]
learn = Learner(dls, model, loss_func=F.cross_entropy, metrics=accuracy, opt_func = Adam, cbs=cbs)

In [ ]:
#|eval: false
learn.fit_one_cycle(15, lr_max=7e-3);